In [1]:
from neo4j import GraphDatabase

root_dir = '/Users/hp/workbench/data/json_dir'

In [5]:

class NeoExec(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def exec(self, stmt):
        with self._driver.session() as session:
            result = session.write_transaction(self._exec, stmt)
            print(result.value(0))

    @staticmethod
    def _exec(tx, stmt):
        result = tx.run(stmt)
        return result


In [6]:
neo = NeoExec(uri="bolt://localhost:7687", user="neo4j", password="password")
ctr = 0